# Activating Reticulate

In [1]:
%load_ext rpy2.ipython
%reload_ext rpy2.ipython

In [2]:
%%R
library(reticulate)
#reticulate::use_condaenv()
reticulate::use_python(python="/usr/bin/python3")
reticulate::use_python(python="/home/ubuntu/anaconda/bin/python3")
print(reticulate::py_config())
#sessionInfo()

python:         /home/ubuntu/anaconda/bin/python3
libpython:      /home/ubuntu/anaconda/lib/libpython3.7m.so
pythonhome:     /home/ubuntu/anaconda:/home/ubuntu/anaconda
version:        3.7.3 (default, Mar 27 2019, 22:11:17)  [GCC 7.3.0]
numpy:          /home/ubuntu/anaconda/lib/python3.7/site-packages/numpy
numpy_version:  1.16.4

python versions found: 
 /usr/bin/python3
 /home/ubuntu/anaconda/bin/python3
 /home/ubuntu/anaconda/bin/python
 /home/ubuntu/anaconda/envs/R/bin/python


# Get the connector to dremio with the class DremioQueryMakerHTTP

In [3]:
%run /home/ubuntu/notebooks/covid-prj01-data-analysis/[UTILS]/dremio-access-new.py
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.max_columns', None) 

dremio = DremioQueryMakerHTTP()
index = dremio.getIndex()
display(index)

QUERY STATUS: PLANNING
QUERY STATUS: COMPLETED


,TABLE_CATALOG,TABLE_NAME,TABLE_SCHEMA,VIEW_DEFINITION
0,DREMIO,brasil_io,covidlake-raw,"SELECT * FROM ""brasil.io"""
1,DREMIO,google_trends,covidlake-raw,"SELECT * FROM ""google_trends.parquet"""
2,DREMIO,health_ministry,covidlake-raw,SELECT * FROM health_ministry
3,DREMIO,hospitals_dimension,covidlake-raw,"SELECT * FROM ""hospitals.csv"""
4,DREMIO,wcota,covidlake-raw,SELECT * FROM wcota
5,DREMIO,VDS-cases-state,PowerBI,"select a.state,a.city, sum(a.confirmed) as cases, ""a""...."
6,DREMIO,dolar_ratio,covidlake-raw.ipea,"SELECT * FROM ""dolar_ratio.parquet"""


# The HTTP connector is fairly slower than the ODBC and JDBC ones because of 500 registries per request hard limit on dremio API

In [4]:
brasil_io = dremio.getTable(0)
display(brasil_io.head())

QUERY STATUS: PLANNING
QUERY STATUS: PLANNING
QUERY STATUS: COMPLETED


,city,city_ibge_code,confirmed,confirmed_per_100k_inhabitants,date,death_rate,deaths,estimated_population_2019,index,is_last,order_for_place,place_type,state
0,None,35.0,1,0.00218,2020-02-25,NaN,0,45919049.0,9813,False,1,state,SP
1,São Paulo,3550308.0,1,0.00816,2020-02-25,NaN,0,12252023.0,12683,False,1,city,SP
2,None,35.0,1,0.00218,2020-02-26,NaN,0,45919049.0,9812,False,2,state,SP
3,São Paulo,3550308.0,1,0.00816,2020-02-26,NaN,0,12252023.0,12682,False,2,city,SP
4,None,35.0,1,0.00218,2020-02-27,NaN,0,45919049.0,9811,False,3,state,SP


# Converting Python DataFrame to R DataFrame

### Use %%R above the cell to use R code on a Python Kernel

In [5]:
%%R

r_brasil_io <- py$index
head(r_brasil_io)

  TABLE_CATALOG          TABLE_NAME  TABLE_SCHEMA
1        DREMIO           brasil_io covidlake-raw
2        DREMIO       google_trends covidlake-raw
3        DREMIO     health_ministry covidlake-raw
4        DREMIO hospitals_dimension covidlake-raw
5        DREMIO               wcota covidlake-raw
6        DREMIO     VDS-cases-state       PowerBI
                                                                                                                                                                                 VIEW_DEFINITION
1                                                                                                                                                                      SELECT * FROM "brasil.io"
2                                                                                                                                                          SELECT * FROM "google_trends.parquet"
3                                                                      

In [7]:
%%R
class(r_brasil_io)

[1] "data.frame"


In [11]:
%%R
dim(r_brasil_io)

[1] 7 4
